# Predicting Taxi Fares in NYC
## Data Cleaning, Visualisation, Feature Engineering

In [ ]:
import gc
from datetime import datetime
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
from typing import Tuple, List, Union, Iterable, NamedTuple
from enum import Enum
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import s2sphere as s2
from geographiclib.geodesic import Geodesic
from matplotlib.path import Path
import utm
import math
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM

from bokeh.io import show, output_notebook
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
    FactorRange,
    Row
)
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.palettes import Spectral, Category20c, Viridis3, BuGn
from bokeh.transform import cumsum
import holoviews as hv #There is a reason we have to do this here but its not important. Holoviews is the next library
hv.extension('bokeh')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier, \
    AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, mean_squared_error, precision_score, recall_score, roc_auc_score
import xgboost as xgb

import lightgbm as lgb

from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix  

%matplotlib inline

### Helper functions and classes

In [ ]:
geoid = Geodesic.WGS84

EARTH_CIRCUMFERENCE_METERS = 1000 * 40075.017

GPSLocation = Tuple[float]
GPSTrace = Tuple[List[float], List[GPSLocation]]

S2_MAX_CELLS=1000
S2_MAX_LEVEL=22
S2_MIN_LEVEL=13

MIN_LAT = -1.0*np.pi/2.0
MAX_LAT = np.pi/2.0

MIN_LON = -1.0*np.pi
MAX_LON = np.pi

MIN_LAT_DEG, MAX_LAT_DEG, MIN_LON_DEG, MAX_LON_DEG = 39, 43, -75, -70

class S2Mode(Enum):
    """Types of s2 cell identifiers"""
    ID = "ID"
    TOKEN = "TOKEN"


class GPS:
    """GPS location information."""

    Delta = NamedTuple("GPSDelta", [("lat", float), ("lon", float)])

    def __init__(self, lat: float=0., lon: float=0., alt: float=0.):
        self.coords = np.array([lat, lon, alt])
        self.coords_radians = np.array(list(map(earth_metres_to_radians, self.coords)))

    def __getstate__(self):
        return {'coords': [self.lat, self.lon, self.alt]}

    def __setstate__(self, state):
        self.coords = np.array(state['coords'])

    def __getattr__(self, item):
        if item == 'lat':
            return self.coords[0]
        elif item == 'lon':
            return self.coords[1]
        elif item == 'alt':
            return self.coords[2]

    def __setattr__(self, key, value):
        if key == 'lat':
            self.coords[0] = value
        elif key == 'lon':
            self.coords[1] = value
        elif key == 'alt':
            self.coords[2] = value
        elif key == 'coords':
            self.__dict__['coords'] = value

    @classmethod
    def interpolate(cls, trace: GPSTrace, x: float):
        """Interpolate GPS location at specified time point in ms since epoche."""
        assert len(trace) == 2
        assert trace[1].ndim == 2
        return cls(
            lon=np.interp(x, trace[0], trace[1][:, 0]),
            lat=np.interp(x, trace[0], trace[1][:, 1]),
            alt=np.interp(x, trace[0], trace[1][:, 2]) if trace[1].shape[1] == 3 else 0.
        )

    def __str__(self):
        return self.to_str()

    def __repr__(self):
        return self.to_str(prefix=type(self).__name__ + "(", suffix=")", keywords=True, precision=9)

    def to_str(self, prefix="GPS(", suffix=")", sep: str=", ", precision: int=6, alt=True, keywords: bool=False):
        coords = [self.lon, self.lat]
        if alt:
            coords.append(self.alt)
        coords = [str(round_sigfigs(coord, precision)) for coord in coords]
        if keywords:
            coords = ['='.join(item) for item in zip(['lon', 'lat', 'alt'], coords)]
        return prefix + sep.join(coords) + suffix

    def to_s2_cell_token(self, level=S2_MAX_LEVEL) -> str:
        latlng = s2.LatLng.from_degrees(self.lat, self.lon)
        return str(s2.CellId.from_lat_lng(latlng).parent(level).to_token())

    @staticmethod
    def from_s2_token(s2_token: str):
        latlng = s2.CellId.from_token(s2_token).to_lat_lng()
        return GPS(lat=latlng.lat().degrees, lon=latlng.lng().degrees)
    
    def __eq__(self, other):
        if not isinstance(other, GPS):
            return False
        return np.allclose(self.coords, other.coords)

    def distance(self, other) -> float:
        """Calculate distance from this to another geographic location in meters."""
        return geoid.Inverse(self.lat, self.lon, other.lat, other.lon)['s12']

    def delta(self, radius: float) -> 'GPS.Delta':
        """Get maximum offset of latitude and longitude coordinates given radius in meters."""
        N = 0.   # azimuth of north direction
        E = 90.  # azimuth of  east direction
        a = geoid.Direct(self.lat, self.lon, N, radius)
        b = geoid.Direct(self.lat, self.lon, E, radius)
        return GPS.Delta(lat=a['lat2'] - a['lat1'], lon=b['lon2'] - b['lon1'])


def earth_metres_to_radians(meters):
    """Return radians on surface of earth for a given length in meters"""
    return (2 * math.pi) * (float(meters) / EARTH_CIRCUMFERENCE_METERS)


def get_s2_covering_region(center: GPS,
                           radius: float,
                           max_level: int=S2_MAX_LEVEL,
                           min_level: int=S2_MIN_LEVEL,
                           max_cells: int=S2_MAX_CELLS,
                           mode: S2Mode=S2Mode.TOKEN) -> List[Union[str, int]]:
    """Given a GPS point and a radius, return the set of S2 IDs of specified level spanning the circle defined by the
    input radius
    Args:
        center: GPS point
        radius: meters
        level: S2 Cell ID level (default is 19)

    Return:
        s2_rect: list of S2 cell IDs
    """
    radius_radians = earth_metres_to_radians(radius)
    latlng = s2.LatLng.from_degrees(float(center.lat), float(center.lon)).normalized().to_point()
    region = s2.Cap.from_axis_height(latlng, (radius_radians*radius_radians)/2)
    coverer = s2.RegionCoverer()
    coverer.min_level = min_level
    coverer.max_level = max_level
    coverer.max_cells = max_cells
    covering = coverer.get_covering(region)
    s2_rect = set()
    for cell_id in covering:
        if mode == S2Mode.TOKEN:
            s2_rect.add(cell_id.to_token())
        elif mode == S2Mode.ID:
            s2_rect.add(cell_id.id())
    return list(s2_rect)

def get_region(lat_rad, lon_rad, region_rad):
    min_lat = lat_rad - region_rad
    max_lat = lat_rad + region_rad
    if min_lat > MIN_LAT and max_lat < MAX_LAT:
        delta_lon = np.arcsin(np.sin(region_rad)/np.cos(lat_rad))
        min_lon = lon_rad - delta_lon
        max_lon = lon_rad + delta_lon
        if min_lon < MIN_LON:
            min_lon += 2.0*np.pi
        if max_lon > MAX_LON:
            max_lon -= 2.0*np.pi
    else:
        min_lat = np.max([min_lat, MIN_LAT])
        max_lat = np.min([max_lat, MAX_LAT])
        min_lon = MIN_LON
        max_lon = MAX_LON
    return min_lat, min_lon, max_lat, max_lon

def round_sigfigs(number: float, decimals: int) -> str:
    """Round floating point value to specified number of fractional digits and convert to string."""
    assert decimals >= 0
    factor = pow(10., decimals)
    result = str(round(number * factor) / factor)
    integral, fractional = result.split(".")
    if decimals < 1:
        return integral
    return ".".join([integral, fractional.ljust(decimals, "0")])

def calculate_distance(row) -> float:
    pickup = GPS(lat=row['pickup_latitude'], lon=row['pickup_longitude'])
    dropoff = GPS(lat=row['dropoff_latitude'], lon=row['dropoff_longitude'])
    return pickup.distance(dropoff)

def plot_pickup_dropoff(pickup: List[GPS], dropoff: List[GPS], radius: Union[float, int]=100.0):    
    imagery = OSM(desired_tile_form='L')
    f, ax = plt.subplots(nrows=1, ncols=2, figsize=(7, 3), dpi=300, subplot_kw={'projection': imagery.crs})
    R = earth_metres_to_radians(radius)*180/math.pi
    colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
    
    for i, gps in enumerate([pickup, dropoff]):
        xymin = s2.LatLng.from_degrees(gps[1].lon-R,gps[1].lat-R)
        xymax = s2.LatLng.from_degrees(gps[2].lon+R,gps[2].lat+R)
        extent = [xymin.lat().degrees, xymax.lat().degrees, xymin.lng().degrees, xymax.lng().degrees]
        ax[i].set_extent(extent)
        ax[0].add_image(imagery, 9, cmap="gray")

In [ ]:
def plot_kde_differentiable(df: pd.DataFrame,
                            props: Iterable[str],
                            titles: Iterable[str],
                            labels: Iterable[str],
                            indices: Iterable[int],
                            x_rot: int=75,
                            figsize: Tuple[int, int]=(7,10),
                            nrows: int=1, ncols: int=1
                           ):
    
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, sharex=False, sharey=False)
    plt.subplots_adjust(wspace = 0.3, hspace = .8)
    
    for ind in range(len(indices) - 1, nrows*ncols):
        if ind >= len(props):
            assert ind < nrows*ncols    
            use_multi_ind = nrows!=1 and ncols!=1
            use_no_index = nrows*ncols==1
            pos_y = ind % ncols
            pos_x = ind // ncols

            if use_multi_ind:
                ha = ax[pos_x, pos_y]
            elif use_no_index:
                ha = ax
            else:
                ha = ax[ind]
            ha.axis('off')
        
    for t, l, ind, p in zip(titles, labels, indices, props):
        
        assert ind < nrows*ncols    
        use_multi_ind = nrows!=1 and ncols!=1
        use_no_index = nrows*ncols==1
        pos_y = ind % ncols
        pos_x = ind // ncols

        if use_multi_ind:
            ha = ax[pos_x, pos_y]
        elif use_no_index:
            ha = ax
        else:
            ha = ax[ind]
        
        g0 = sns.kdeplot(df.loc[(df['TARGET'] == 0) & (df[p] > -999), p], shade=True, color=Viridis3[0])
        
        ha.set_title(t)
        ha.set(xlabel=l, ylabel='Density')



### Load data and have a peek

In [ ]:
train_df = pd.read_csv("../data/new-york-city-taxi-fare-prediction/train.csv.zip")
print(train_df.columns)
train_df.head(20)

In [ ]:
total = train_df.isnull().sum().sort_values(ascending = False)
percent = (train_df.isnull().sum()/train_df.isnull().count()*100).sort_values(ascending = False)
missing_train_df  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_train_df.head(3)

### Assumptions for building the model

* Trip distance: increase in distance travelled should be reflected in an increased price
* Time of travel: peak travel times will cost hihger fares
* Day of travel: there may exist a difference in fares on weekends versus weekdays
* Weather conditions: averse weather may lead to greater demand i.e. higher fares
* Airport trips: these usually have fixed fares
* Area: different neighborhoods might have different associated pick-up or drop-off costs
* Availability: taxi availability will of course affect fares

We can plot distributions of the various columns and see if they differentiate well between the fare amounts

In [ ]:
def compute_distance_pickup_dropoff(row):
    """Compute the distance between the pickup and dropoff"""
    pickup = GPS(lat=row['pickup_latitude'], lon=row['pickup_longitude'])
    dropoff = GPS(lat=row['dropoff_latitude'], lon=row['dropoff_longitude'])
    return pickup.distance(dropoff)

def get_mean_min_max_GPS(df: pd.DataFrame, lat: str, lon: str) -> List[GPS]:
    """Stats on pickup and dropoff"""
    lat_mean, lon_mean = df.loc[(df[lat]>=MIN_LAT_DEG) & (df[lat]<=MAX_LAT_DEG) & (df[lon]>=MIN_LON_DEG) & (df[lon]<=MAX_LON_DEG)][[lat, lon]].mean()
    lat_min, lon_min = df.loc[(df[lat]>=MIN_LAT_DEG) & (df[lat]<=MAX_LAT_DEG) & (df[lon]>=MIN_LON_DEG) & (df[lon]<=MAX_LON_DEG)][[lat, lon]].min()
    lat_max, lon_max = df.loc[(df[lat]>=MIN_LAT_DEG) & (df[lat]<=MAX_LAT_DEG) & (df[lon]>=MIN_LON_DEG) & (df[lon]<=MAX_LON_DEG)][[lat, lon]].max()
    
    mean_GPS = GPS(lat=lat_mean, lon=lon_mean)
    min_GPS = GPS(lat=lat_min, lon=lon_min)
    max_GPS = GPS(lat=lat_max, lon=lon_max)
    
    return [mean_GPS, min_GPS, max_GPS]

In [ ]:
train_df_nonZeros = train_df.fillna(-999)
train_df_nonZeros = train_df_nonZeros.loc[(train_df_nonZeros['fare_amount']>0) & 
                                          (train_df_nonZeros['pickup_latitude']!=0) & (train_df_nonZeros['pickup_longitude'] != 0) &
                                          (train_df_nonZeros['dropoff_latitude']!=0) & (train_df_nonZeros['dropoff_longitude'] != 0) &
                                          (train_df_nonZeros['pickup_latitude']>=MIN_LAT_DEG) & (train_df_nonZeros['pickup_latitude']<=MAX_LAT_DEG) & 
                                          (train_df_nonZeros['pickup_longitude']>=MIN_LON_DEG) & (train_df_nonZeros['pickup_longitude']<=MAX_LON_DEG) &
                                          (train_df_nonZeros['dropoff_latitude']>=MIN_LAT_DEG) & (train_df_nonZeros['dropoff_latitude']<=MAX_LAT_DEG) & 
                                          (train_df_nonZeros['dropoff_longitude']>=MIN_LON_DEG) & (train_df_nonZeros['dropoff_longitude']<=MAX_LON_DEG)
                                         ]
gc.enable()
del train_df
gc.collect()

In [ ]:
train_df_nonZeros.head(20)

In [ ]:
pickup_mean_GPS, pickup_min_GPS, pickup_max_GPS = get_mean_min_max_GPS(train_df_nonZeros, 'pickup_latitude', 'pickup_longitude')
dropoff_mean_GPS, dropoff_min_GPS, dropoff_max_GPS = get_mean_min_max_GPS(train_df_nonZeros, 'dropoff_latitude', 'dropoff_longitude')

In [ ]:
print("Mean pickup:\t", pickup_mean_GPS)
print("Min pickup:\t", pickup_min_GPS)
print("Max pickup:\t", pickup_max_GPS)  
print("Mean dropoff:\t", dropoff_mean_GPS)
print("Min dropoff:\t", dropoff_min_GPS)
print("Max dropoff:\t", dropoff_max_GPS)

In [ ]:
# Missing data? Should be 0 now
total = train_df_nonZeros.isnull().sum().sort_values(ascending = False)
percent = (train_df_nonZeros.isnull().sum()/train_df_nonZeros.isnull().count()*100).sort_values(ascending = False)
missing_train_df_nonZeros  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_train_df_nonZeros.head(3)

In [ ]:
# New column with distances
train_df_nonZeros['distance'] = train_df_nonZeros.apply(compute_distance_pickup_dropoff, axis=1)
train_df_nonZeros.head(20)

In [ ]:
train_df_nonZeros.dtypes

In [ ]:
plot_pickup_dropoff([pickup_mean_GPS, pickup_min_GPS, pickup_max_GPS], 
                    [dropoff_mean_GPS, dropoff_min_GPS, dropoff_max_GPS], 
                    20)

In [ ]:
orig_lon,orig_lat,circle_rad,big_rad = -0.04813968113126384,51.531259899256995,25,200

rad = 1.667*big_rad
region_rad = earthMetersToRadians(rad)
lat_rad, lon_rad = np.pi/180.0*orig_lat, np.pi/180.0*orig_lon
min_lat, min_lon, max_lat, max_lon = get_region(lat_rad, lon_rad, region_rad)
xymin = s2.LatLng.from_radians(min_lat, min_lon)
xymax = s2.LatLng.from_radians(max_lat, max_lon)

colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)

# Sort colors by hue, saturation, value and name.
by_hsv = sorted((tuple(mcolors.rgb_to_hsv(mcolors.to_rgba(color)[:3])), name)
                for name, color in colors.items())
sorted_names = [name for hsv, name in by_hsv]

n = len(sorted_names)

col = "#006400"
edge_col = "#00A591"
marker_size = 7
theta = np.linspace(0, 2 * np.pi, 100)
circle_verts = np.vstack([np.sin(theta), np.cos(theta)]).T
concentric_circle = Path.make_compound_path(Path(circle_verts[::-1]),
                                        Path(circle_verts * 0.6)) 

imagery = OSM()
f = plt.figure(figsize=(7, 7), dpi=300)
ax = plt.axes(projection=imagery.crs)

# Define extent of map to display 
c_lat_rad, c_lon_rad = np.pi/180.0*orig_lat, np.pi/180.0*orig_lon
circle = get_region(c_lat_rad, c_lon_rad, earthMetersToRadians(big_rad))
circle_w = 180.0*np.abs(circle[1]-circle[3])/np.pi
circle_h = 180.0*np.abs(circle[0]-circle[2])/np.pi
ax.set_extent([xymin.lng().degrees, xymax.lng().degrees,
       xymin.lat().degrees, xymax.lat().degrees])
ax.add_patch(mpatches.Ellipse(xy=[orig_lon,orig_lat], width=circle_w, height=circle_h, facecolor='#e2ffff',alpha=0.75,
    transform=ccrs.Geodetic()
    ))

ax.add_image(imagery, 17, cmap='gray')